In [1]:
import numpy as np
import pandas as pd
import pickle
#path stuff
from pathlib import WindowsPath, PosixPath
import sys
PathPath = WindowsPath if 'win' in sys.platform.lower() else PosixPath
PARENT_DIR = PathPath('../')
INFERENCE_PATH = PARENT_DIR.joinpath('.inference')  
GIT_TABLE_PATH = PARENT_DIR.joinpath('.git_tables')
sys.path.append(str(PARENT_DIR))
from common_util import _FILE_TO_LABEL as file_to_label,markdown_table_from_df

### Extract Model Inadequcies for Comparison
compare the calibrated model inadquency between the combined inference and independent inference to evaluate the impact of considering each of the datasets together.

In [4]:
model_name = 'JMAK_{}'

data_names = [v for k,v in file_to_label.items() if 'shah' not in k.lower()]

sigma = []
for data_name in data_names:
    with open(INFERENCE_PATH.joinpath(f'{model_name.format(data_name)}_trunc_normal_samples.pkl'),'rb') as f:
        samples = pd.DataFrame.from_dict(pickle.load(f))
    sigma.append(samples['sigma'].mean())

individual_sigma = pd.Series(sigma,index=data_names,
                             name = 'individual')

with open(INFERENCE_PATH.joinpath(f'{model_name.format("hierarchical")}_trunc_normal_samples.pkl'),'rb') as f:
    posterior_samples = pickle.load(f)

combined_sigma = pd.Series(posterior_samples['sigma'].mean(axis=0),index=data_names,
                           name = 'combined')

model_error = pd.concat([individual_sigma,combined_sigma],axis=1)
table = markdown_table_from_df(np.round(model_error,4),
                               'Independent vs. Combined Error Comparison',
                               'Recrystallization model inadequecy standard deviation comparison across datasets')

with open(GIT_TABLE_PATH.joinpath('model_error_comparison_truncated_normal.tbl'),'w') as f:    
    f.write(table)
